<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself.<br>
<b>Name</b>: Veton Abazovic<br>
<b>NetID</b>:va187<br>
<b>Recitation (01/02)</b>: 02<br>
<b>Notes to Grader</b> (optional):<br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>:    VA 


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

# Final Project: Deep Learning with TensorFlow

** This final project is due Thursday May 10, 2019 at 11:59pm (graded on accuracy and completeness) **

In this project we will work through the process of:
1. installing TensorFlow in your local environment
2. Learn some basics of how to use TensorFlow API
3. Implement a classical linear regression model with TensorFlow
4. Implement a k-means for clustering images
5. Import and Train a model with FashionNIST dataset 
6. Predict new images based on the trained model

This project should give you some basic knowledge on how to work with deep learning systems. Although we do not implement any neural networks from scratch in this lab, it is also important for you to have an understanding of how neural networks work.Please refer to classnotes to see how neural networks are trained. In this project we will use existing libraries, but by selecting and tuning parameters.

# Part 0 - Install TensorFlow
This is our first experience with deep learning methods. We need to set things up first. Execute the lines below to get the setup going. Set up only needs to be done once.

In [ ]:
# Task 0.1. tf requires a numpy update
!pip install numpy --upgrade

In [ ]:
# Task 0.2. install tensorflow
!pip install --user tensorflow 
# you can also specify what version of TF you need.

In [ ]:
# Task 0.3. install keras API
!pip install --user keras

# Initialize 

In [ ]:
# # Task 0.4. all the import we need
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras     # we can also access keras API with tf.keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
# now we are ready to move on to doing some amazing things.

# Part 1 - Getting to know TensorFlow (TF)
TensorFlow is a framework. This core open source library help you develop and train ML models. You can install tensorflow locally or run tensorflow on colab.research.google.com directly in your browser. TensorFlow supports wide range of devices and API's, including javascript for deploying mobile ML based applications. It can run on single CPU systems, GPUs, TPU's as well as mobile devices and large scale distributed systems of hundreds of machines.
TensorFlow is its own framework. Everything in TF is implemented as a computational graph. The graph nodes are executed at later times. 
Let us understand some of its basic API functionality.

## Task 1.1 Learn the basics
TensorFlow is a framework to allow efficient computing with multi-dimensional data. All the calls to TF are executed through API calls. It uses python (altough there are other TF suported languages now such as R)  in the backend to exceute these commands. All objects are stored as tensors, a form of a vector. A TF graph can be evaluated using a tensorflow session. Here are few examples. Just run the cells and see what happens. 

In [ ]:
# create two constant tensors
two_node = tf.constant(8)     
three_node = tf.constant(9)
# sum of the two tensors
sum_node = two_node + three_node    # need a session to evaluate the value of sum_node
# product the two objects
pdt_node = two_node * three_node
print(sum_node)                   # should notice that they are just unevaluated tensors
# A session is an object instance that encapsulate the state and operations on the object.
with tf.Session() as sess:
    # call run method in the session object to evaluate the two_node and sum_node objects
    print(sess.run([sum_node, pdt_node]))
    
# QUESTION : Explain why print(sum_node) does not show the value 17
#  It is because its a tensor object instead of a value

## Loops in TensorFlow
 read more about loops in : https://www.tensorflow.org/api_docs/python/tf/while_loop

In [ ]:
# here is some sample code on how to write loops. Just modify the code
i = tf.constant(0)
cond = lambda i: tf.less(i, 12)
body = lambda i: tf.add(i, 1)
r = tf.while_loop(cond, body, [i])   # returns the final value of i
with tf.Session() as sess:
    print(sess.run(r))



### Another Example of a While Loop

In [ ]:
# some other demo code on how to use tf.while
def cond(t1, t2):
    return tf.less(t1, t2)

def body(t1, t2):
    return [tf.add(t1, 1), t2]

t1 = tf.constant(2)
t2 = tf.constant(4)

res = tf.while_loop(cond, body, [t1, t2])

with tf.Session() as sess:
    print(sess.run(res))

# question: What happens if we change both t1 and t2 at the same time?
#If they are swapped then the addition does not occur

## Task 1.2 Find the first 10 Fibbonachchi numbers
In this activity we will write TF code using tensorflow framework to output the first 10 Fibonnachi numbers 1 1 2 3 5 8 13 21 34 55 as a list

In [ ]:
## BEGIN SOLUTION
# define the first 2 terms of the sequence
a = tf.Variable(0)
b = tf.Variable(1)

newterm = tf.add(a,b)     

with tf.control_dependencies([newterm]):
    update1 = tf.assign(a, b)

    with tf.control_dependencies([update1]):
        update2 = tf.assign(b, newterm)
#initializes
init = tf.global_variables_initializer()

# run
with tf.Session() as sess:
    sess.run(init)
    fibonacci = [a.eval(), b.eval()]
    for i in range(9):
         next, up1, up2 = sess.run([newterm, update1, update2])
         fibonacci.append(next)
    
    #print(type(fibonacci))
    fibonacci.pop(0)
    print(fibonacci)

## END SOLUTION

## Task 1.3 Activating Placeholders
In the execution of the tensorflow objects, it is possible that you will have to wait for some results to show after a cycle of operations. A placeholder is more like a variable that you would declare in a program and initialize later. In general, Insert a placeholder for a tensor that will be always fed.We can use tf.placeholder to make calls to placeholder tensors. Read more about placeholders at https://www.tensorflow.org/api_docs/python/tf/placeholder

In [ ]:
## BEGIN SOLUTION

# two tensor placeholders that can hold int32 data
x = tf.placeholder(tf.int32, name='x')       # x can hold int32 
y = tf.placeholder(tf.int32,name='y')       # y can hold int32 

# Find sum, difference and product
sumOf = tf.add(x,y)            # use tf.sum
diff =  tf.subtract(x,y)            # use tf.subtract
prod =  tf.multiply(x,y)            # use tf.multiply

# store inputs as a dictionary
dict_values = {x:20,y:30}          # store values x = 20 and y = 30 as a dictionay

## END SOLUTION

# run a session
with tf.Session() as sess:
    print('Sum',sess.run(sumOf,feed_dict=dict_values))
    print('Difference',sess.run(diff,feed_dict=dict_values))
    print('Product',sess.run(prod,feed_dict=dict_values))

## Task 1.4 Matrix Operations
The power of the concept of a tensor is its ability to treat all data as vectors. This makes it easy to parallelize many operations such as matrix multiplication. If you run code on colab.research.google.com, you can also choose some server configuration (GPU, TPU) to run your code faster.

In [ ]:
## BEGIN SOLUTION

import numpy as np
# initialize numpy arrays a and b
a = np.array([[5.0,5.0]])               
b = np.array([[2.0],[2.0]])  

# define two tensors mat1 and mat2 using numpy arrays a and b
mat1 = tf.convert_to_tensor(a)                # mat1 is a rank-1 tensor               
mat2 = tf.convert_to_tensor(b)                # mat2 is a rank-2 tensor  

# build a matrix multiplier tensor
matrix_pdt = tf.matmul(mat1, mat2)    # use tf.matmul to find the result
# run a session and perform the operartions

## END SOLUTION

with tf.Session() as sess:
    result = sess.run(matrix_pdt)
    print(result)

### Read API Docs
The above examples were just few things to demonstrate that tensorflow is a complete framework and one can code an entire program using tensor objects. Read more at https://www.tensorflow.org/api_docs

# Part 2 Handwriting classification with k-means
In this part, we will use the MNIST dataset to import 60,000 handwritten digits and implement k-means to classify them. MNIST is a benchmark dataset that can help learn many things about Deep Learning basics.
This Dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images. You can import this data from keras.datasets or from tensorflow.examples.tutorials.mnist. You can use either form in your solutions.

In [ ]:
## import from kears. The data will be downloaded as numpy.ndarray
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# plot images to visualize
import matplotlib.pyplot as plt
%matplotlib inline
plt1 = plt.imshow(x_train[0].reshape(28,28))   # visualize image[1]

## Task 2.1 import data from MNIST
In this task we will import the MNIST digits dataset and learn the properties of the dataset

In [ ]:
import tensorflow as tf
# get the MNIST Dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
type(mnist)

## Task 2.2 Find Stats
Find how many training and validation images are in the set

In [ ]:
# BEGIN SOLUTION
# how many train images are in the dataset?
train_images = len(mnist.train.labels)           
print("train images: ", train_images)
# how many validation images are in the dataset?
validation_images = len(mnist.validation.labels) 
print("validation images: " , validation_images)
# END SOLTUTION

## Task 2.3 Visualize the image
Each image is a gray-scale 28x28 (784 pixles) image. You can treat the image as a rank-1 tensor (or 1D vector) of size 784 with each value between 0-255. We can plot some images here.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt1 = plt.imshow(mnist.train.images[2].reshape(28,28))   # visualize image[1]

## Task 2.4 Implementing k-means algorithm
In this part we will implement k-means algorithm to identify possible clusters in this data. An image in this data set is a gray-scale 28 x 28 image (784 pixels with values from 0-255). We will use the eucledean distance between two images to see the "distance" between them. Since we are working in high dimension (784) we can only understand the concept of distance here. We will use the k-means algorithm that was discussed in lecture 12.2 slide 15. You do not need to use TF modules t do this.


### Task 2.4.1 Implement the k-means algorithm (vectorized)

In [ ]:
# The vectorized version of k-means is given on the slide 15 of Lecture 12.2.
# Just use the same code.

## BEGIN SOLUTION
def kmeans(X, k, max_iter=10, rand_seed=0):
    np.random.seed(rand_seed)
    Mu = X[np.random.choice(X.shape[0],k),:]
    for i in range(max_iter):
        D = -2*X@Mu.T + (X**2).sum(axis=1)[:,None] + (Mu**2).sum(axis=1)
        y = np.argmin(D,axis=1)
        Mu = np.array([np.mean(X[y==i],axis=0) for i in range(k)])
    loss = np.linalg.norm(X - Mu[np.argmin(D,axis=1),:])**2/X.shape[0]
    return Mu, y, loss


## END SOLUTION

### Task 2.4.2 Cluster MNIST images.
Experiment to Find a good value of k such that most centers look like images.

In [ ]:
## BEGIN SOLUTION

# start with k = 50
k = 50
# flatten each image to be 784 value vector and form 55000 x 784 matrix X
X = mnist.train.images.reshape(55000, 784)

# Choose k random centers from X. That is k random data points become the original cluster centers
Mu = X[np.random.choice(X.shape[0], k), ::]
# call the kmeans algorithm defined above. Be sure to have the proper arguments passed to kmeans.
[Mu, y, loss] = kmeans(X, k)

# Plot the cluster centers as images

fig, ax = plt.subplots(10, 5, figsize=(10, 10))

centers = Mu.reshape(50, 28, 28)
#loop
for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)
    
## END SOLUTION

## Task 2.5 (optional for Extra Credit)
Implement the k-means++ algorithm to see if better clusters can be obtained in fewer iterations. This task is optional and you can add cells below to perform the operations. Do this part after completing the all the parts below.

In [ ]:
## BEGIN SOLUTION (OPTIONAL)

## END SOLUTION

# Part 3 Linear Regression with TensorFlow
We have learned how to implement linear regression using Pandas. In this exercise, we will learn how to use tensorflow to implement linear regression. As stated TensorFlow is a framework that allows you to implement many standard ML algorithms. We will implement Linear Regression using TF libraries.

## Task 3.1 set up
Let us set up basic things needed to get tensorflow going. You are asked to experiment with different values of alpha (learning rate), epochs (how many rounds in training. Too little is under-fitting, too much is over-fitting). You need to display the process every 50 steps.

In [ ]:
# initialize
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt
rng = numpy.random

# BEGIN SOLUTION
# set parameters
alpha = 0.01    # alpha is the learning rate
epochs = 1000   # One epoch is when an entire dataset is passed both forward and backward through the neural network 
                # only once
step = 50       # every 50 iterations display the results.
# END SOLUTION

## Task 2.2 Read data
We will work with our CS 205 Data set. First we create a data frame, then extract midterm and final exam scores as numpy arrays.

In [ ]:
# Read the Training Data

# BEGIN SOLUTION
import pandas as pd
df = pd.read_csv("data/midterm_finals_CS205F18.csv")

# use first 110 records for the training data
train_X = df.iloc[0:110,[0]].values
train_Y = df.iloc[0:110,[1]].values

# use the remaining for validation
validate_X = df.iloc[110:,[0]].values
validate_Y = df.iloc[110:,[1]].values

n_samples = train_X.shape[0]
print("training set size: ", n_samples) 
print("validation set size: ", validate_X.shape[0])    
# END SOLUTION

## Task 2.3 Intialize model
Set model weights and bias. You may want to try random weights and random bias. If they do not provide a good regression line, try to pre-set some values for weight and bias. The values must be floats.

In [ ]:
# tf Graph Input. We will feed the values later using feed_dict.
X = tf.placeholder("float")
Y = tf.placeholder("float")

# BEGIN SOLUTION
# Variables are manipulated via the tf.Variable class. A tf.Variable represents a tensor 
# whose value can be changed by running ops on it. tf.Variable exists outside the context of a single session.run call.

theta = tf.Variable(1.0, tf.float32)
b = tf.Variable(1.0, tf.float32)
# END SOLUTION

## Task 2.4 Linear Model

In [ ]:
# Construct a linear model using tf.add and tf.multiply. The model provides y = theta*x + bias
# BEGIN SOLUTION
predictor = tf.add(tf.multiply(X, theta), b)
# END SOLUTION

In [ ]:
# Compute the average squared error using TF calls
# BEGIN SOLUTION
cost = tf.reduce_sum(tf.pow(predictor-Y, 2)) / (2 * n_samples)
# Use the Gradient descent Optimizer from tf.train
optimizer = tf.train.GradientDescentOptimizer(alpha).minimize(cost)
# END SOLUTION

In [ ]:
# Initialize the variables (i.e. assign their default value)
## BEGIN SOLUTION
init = tf.global_variables_initializer()
## END SOLUTION

## Task 2.5 Train the Model
Train the linear regression model by running the training data through the network.

In [ ]:
with tf.Session() as sess: 
      
    # Initializing the Variables 
    sess.run(init) 
      
    # Iterating through all the epochs 
    for epoch in range(epochs): 
          
        # Feeding each data point into the optimizer using Feed Dictionary 
        for (_x, _y) in zip(train_X, train_Y): 
            sess.run(optimizer, feed_dict = {X : _x, Y : _y}) 
          
        # Displaying the result after every 50 epochs 
        if (epoch + 1) % 50 == 0: 
            # Calculating the cost a every epoch 
            c = sess.run(cost, feed_dict = {X : train_X, Y : train_Y}) 
            print("Epoch", (epoch + 1), ": cost =", c, "theta =", sess.run(theta), "b =", sess.run(b)) 
      
    # Storing necessary values to be used outside the Session 
    training_cost = sess.run(cost, feed_dict ={X: train_X, Y: train_Y}) 
    Weight = sess.run(theta) 
    bias = sess.run(b) 
    
## END SOLUTION

## Task 2.6 Display Regression Line

In [ ]:
#Graphic display
with tf.Session() as sess:
    sess.run(init)
    plt.plot(train_X, train_Y, 'ro', label='Original data')
    plt.plot(train_X, sess.run(theta) * train_X + sess.run(b), label='Fitted line')
    plt.legend()
    plt.show()

## Task 2.7 Compare with Validation Data
We have some validation samples and let us see where the line fits

In [ ]:
## BEGIIN SOLUTION

with tf.Session() as sess:
    sess.run(init)
    plt.plot(validate_X, validate_Y, 'ro', label = 'Original Data')
    plt.plot(validate_X, sess.run(theta) * validate_X + sess.run(b), label = 'Fitted Line')
    plt.legend()
    plt.show()
## END SOLUTION

# Part 3 - First Exercise in Deep Learning
Deep Learning has become quite popular in recent times (since 2012) due to its remarkable advances in applications including self driving cars, image recognition, and voice to text conversion. The hello world of deep learning (DL) is long considered to be the exercise that uses of the MNIST hand-written digits dataset that was created by Yann Lecun in 1998. Yann LeCunn who is a professor at NYU and Director of AI at Facebook, just won the 2019 Turing Award for his contributions to Deep Learning. Let us go through a series of exercises to learn how to read, train and validate an image data set. In this case the 60,000 handwritten images of 0-9.

## Task 3.1 download the MNIST dataset

In [ ]:
import tensorflow as tf
# get the MNIST Dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

In [ ]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Task 3.2 Learn about the data set.
Just write some code to answer the following questions. 

- how big is the training set?
- how big is the test set?

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
# read the MNIST dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# get training images from MNIST
train_images = mnist.train.images
# how big is test images?
test_images = mnist.test.images.shape[0]
# how big is validation set?
validation_images =mnist.validation.images.shape[0]
### END SOLUTION
print("\ntest images:", test_images, "validation images:", validation_images)

## Task 3.3 Visualize the data set
Each image is a 28 x 28 gray-scale 2D image. Display the images images[0] and images[1] as 2D images.
Then flatten and display image[0] to understand the form of the data.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt1 = plt.imshow(mnist.train.images[0].reshape(28,28))
plt2 = plt.imshow(mnist.train.images[1].reshape(28,28),cmap='gist_gray')  # gray scale
# flatten images[1] and display
plt3 = plt.imshow(mnist.train.images[1].reshape(784,1),cmap='gist_gray',aspect=0.02)

# show first 4 images side by side. 
f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(mnist.train.images[0].reshape(28,28))
axarr[0,1].imshow(mnist.train.images[1].reshape(28,28))
axarr[1,0].imshow(mnist.train.images[2].reshape(28,28))
axarr[1,1].imshow(mnist.train.images[3].reshape(28,28))


## Task 3.4 Create a Model

In [ ]:
import tensorflow as tf
## BEGIN SOLUTION

# create a placeholder
x = tf.placeholder(tf.float32,shape=[None,784])  # 784 = mnist.train.images[1].shape

# there are 10 possible numbers that the classifier must recognize
# start with a matrix 784x10 of all zeros
thetas = tf.Variable(tf.zeros([784,10]))

# create a tf.Variable of 10 zeros (we need to find 10 categories 0-9)
b = tf.Variable(tf.zeros([10]))

# Create the Graph thetas*x + b
y = tf.matmul(x, thetas)+b

# loss and optimizer
y_true = tf.placeholder(tf.float32,[None,10])

# Cross-entropy loss, or log loss, measures the performance of a classification model whose output is a 
# probability value between 0 and 1.
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y))

# optimizer is a Gradient Descent Optimizer
optimizer = tf.train.GradientDescentOptimizer(0.5)

# training model using optimizer.minimize. Pass cross_entrophy as the argument
train = optimizer.minimize(cross_entropy)

## END SOLUTION

## Task 3.5 Create a Session and Train

In [ ]:
# initialize all global variables
init = tf.global_variables_initializer()

In [ ]:
# Train the model for 1000, 2000,.., 10000 steps on the training set
# Using built in batch feeder from mnist for convenience

## BEGIN SOLUTION
with tf.Session() as sess:
    sess.run(init)

    for r in range(1000,11000,1000):
        for step in range(r):
            batch_x , batch_y = mnist.train.next_batch(100)
            sess.run(train,feed_dict={x:batch_x,y_true:batch_y})

        matches = tf.equal(tf.argmax(y,1),tf.argmax(y_true,1))
        acc = tf.reduce_mean(tf.cast(matches,tf.float32))
        print("trained in steps", r, sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels}))
   

## END SOLUTION

## Task 3.6 (challenge task - optional)
What is the maximum accuracy you observed in the previous part?  The best models can get closer to 99% accuracy. If you'd like,  you can try to improve the accuracy of the classifier. Please try this after completing all the required activities.

In [ ]:
## Code and documentation for challenge task (optional). You can add any additional cells as needed.

# Part 4 - Classification of Fashion images

## Task 4.1 Set up

In [ ]:
# import the fashionMNISt data set and separate training and test sets
# use keras.datasets.fashion_mnist
from tensorflow import keras
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()  

## Task 4.2 Explore this data set
Inspect the dataset. The class names are not included with the data. We will add them.
'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'

In [ ]:
# add classnames label images (ints) corresponds to thses labels. For example label 0 is T-shirt/top
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# how many train images and their dimensions

## BEGIN SOLUTION
train_images.shape, test_images.shape, test_labels.shape, train_labels.shape

## END SOLUTION

## Task 4.3 Visualize images in the data set

In [ ]:
# visualize images in this data set
## BEGIN SOLUTION
img_number = 10000
plt.figure()
plt.imshow(train_images[img_number])
plt.colorbar()
plt.grid(False)
plt.show()
print(class_names[train_labels[img_number]])

## END SOLUTION

## Task 4.4 Plotting images

In [ ]:
# scale the images
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# plot the first 25 images and their class_names in a 5x5 grid 
## BEGIN SOLUTION
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()
## END SOLUTION

## Task 4.5 Model Creation and Compilation
Read about keras model creation at https://keras.io/models/about-keras-models/
also read about sequential models at https://keras.io/models/sequential/

In [ ]:
# create a keras.sequential model
# use tf.nn.relu as activation. Read more at https://www.tensorflow.org/api_docs/python/tf/nn
# use one input layer (784), one hidden layer(128), and one output layer(10) in the model.
# we are only going to use one hidden layer in the model
# see some examples here: https://keras.io/models/model/

## BEGIN SOLUTION
model = keras.Sequential([keras.layers.Flatten(input_shape= (28,28)), keras.layers.Dense(128, activation = tf.nn.relu), keras.layers.Dense(10, activation = tf.nn.softmax)])

## END SOLUTION

In [ ]:
# compile the model. See https://keras.io/models/model/.
# you can use the adam optimizer. 
# The choice of optimization algorithm for your deep learning model is really important
# The Adam optimization algorithm is an extension to stochastic gradient descent 
# that is better for deep learning applications in computer vision.
## BEGIN SOLUTION

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
## END SOLUTION

## Task 4.6 Fit the model
Fit the model to training images. https://keras.rstudio.com/reference/fit.html
You can use any number of epochs (eg. 10). You should see the accuracy increasing and loss decreasing for training images.

In [ ]:
## BEGIN SOLUTION
model.fit(train_images, train_labels, epochs = 5)

## END SOLUTION

## Task 4.7 Test Accuracy
Using model.evaluate (see https://keras.io/models/model/) find the test_loss and test_accuracy

In [ ]:
## BEGIN SOLUTION
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Testing Accuracy... ', test_acc)
print('Testing Loss... ', test_loss)
## END SOLUTION

## Task 4.8 Predicting
using model.predict, create a prediction model for test_images

In [ ]:
## BEGIN SOLUTION
# create a prediction model for the test images
predictions = model.predict(test_images)
predictions
## END SOLUTION

In [ ]:
## Check to see the accuracy of model prediction
# how well the model predicted an image? 
## BEGIN SOLUTION
image_num = 20
predict = class_names[np.argmax(predictions[image_num])]

print("This Test Image ", image_num, predict)

plt.figure()
plt.imshow(test_images[image_num].reshape(28,28), cmap = plt.cm.binary)
plt.show()

## END SOLUTION

## Task 4.9 Plotting the Predictor confidence
Use the following function plot_image to plot the predictor confidence.

In [ ]:
# code by François Chollet
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# use the plot_image to display images and how well they were predicted.
## BEGIN SOLUTION
i = 0
plt.figure(figsize = (6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)
plt.show()


i = 12
plt.figure(figsize = (6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)
plt.show()

## END SOLUTION

## Task 4.10
Plot a 20 images (shown as 5x4 grid) to see test images and their confidence

In [ ]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
## BEGIN SOLUTION
rows = 5
cols = 4
imgs = rows * cols
plt.figure(figsize=(2*2*cols, 2*rows))
for i in range(imgs):
    plt.subplot(rows, 2*cols, 2*i+1)
    plot_image(i, predictions, test_labels, test_images)
    plt.subplot(rows, 2*cols, 2*i+2)
    plot_value_array(i, predictions, test_labels)
plt.show()


## END SOLUTION

## Extra Credit
Find the image with the least confidence above and re-train the network to see if you can improve the prediction accuracy to over 90%. You should not lose too much from the confidence in others.

### Feedback
Please provide feedback on this lab.
* how would you rate this lab (from 1-10, 10-highest) : 8
* how can we improve his lab? :N/A

<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> File Name:</b> Please name the file as your_section_your_netID_finalProj.jpynb<br>
<b> Submit To: </b> Canvas &rarr; Assignments &rarr; finalProj <br>
<b>Warning:</b> Failure to follow directions may result in loss points.<br>
</div>

Lab designed by A.D. Gunawardena, 2019. Acknowledgements: Google TensorFlow group